## Assign topics to each tweet using the best iteration of each model

In [1]:
import tweetclassifier as tc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime
from time import time

%matplotlib inline
warnings.filterwarnings('ignore')

pd.set_option('display.max_info_columns',200)
pd.set_option('display.max_colwidth',200)
pd.set_option('display.max_rows', 100)
plt.style.use('ggplot')

In [2]:
clf = tc.classifytweet()

Model items loaded and classifier initialized!


In [3]:
df = pd.read_csv('../data/data_cleaned_w_sentiment_outrage_counts.csv', sep=',', quotechar='"', index_col=False, header=0)
df.drop(['text_token','text_stemmed','text_processed', 'text_tokenized'], inplace=True, axis=1)
df.head(1)

,text,topic,log_followers,log_retweets,engagement_flag,positive_emo,outrage_emo,net_emo_outrage,predicted_sentiment,negative_sentiment_prob,num_outrage_words_ext,num_outrage_words
user.description,,,,,,,,,,,,
"Part of the #VastRightWingConspiracy -- thanks, @HillaryClinton. Better candidate: https://www.randpaul.com/ Retweeting not endorsement. Image: TV lover.","#BanTheBox means ""all is #forgiven""/NO MORE federal #NCIC checks when buying guns? Please consider it, Mr. President @POTUS. #forgiving",G,5.252273,0.0,0,0,0,0,4,0.265578,0,0


### Get the outrage index parameters and scale the scores to [0,1]

In [4]:
#df['base_outrage_count'] = 0
#df['enhanced_outrage_count'] = 0
df['arousal'] = 0
df['valence'] = 0
df['sentiment'] = 0
#df['emoji_count'] = 0
df['topics'] = np.empty((len(df), 0)).tolist()
df['topic_arousal'] = 0
df['topic_valence'] = 0
df['outrage'] = 0

In [19]:
df.head(1)

,text,topic,log_followers,log_retweets,engagement_flag,positive_emo,outrage_emo,net_emo_outrage,predicted_sentiment,negative_sentiment_prob,num_outrage_words_ext,num_outrage_words,arousal,valence,sentiment,topics,topic_arousal,topic_valence,outrage
user.description,,,,,,,,,,,,,,,,,,,
"Part of the #VastRightWingConspiracy -- thanks, @HillaryClinton. Better candidate: https://www.randpaul.com/ Retweeting not endorsement. Image: TV lover.","#BanTheBox means ""all is #forgiven""/NO MORE federal #NCIC checks when buying guns? Please consider it, Mr. President @POTUS. #forgiving",G,5.252273,0.0,0,0,0,0,4,0.265578,0,0,0,0,0,[],0,0,0


In [10]:
col_map = {v:ix for ix,v in enumerate(df.columns.tolist())}
tweets = df.text.tolist()
to_insert = list()
t0 = time()
for ix in range(df.shape[0]):
    if ix % 2500 == 0:
        elapsed = time() - t0
        print("Processing tweet number %s, time elapsed: %s" %(str(ix+1), str(datetime.timedelta(seconds=elapsed))))
    clf.prepare_tweet(tweets[ix])
    '''
    #df.iloc[ix, col_map['base_outrage_count']] = clf.get_base_outrage_count()
    #df.iloc[ix, col_map['enhanced_outrage_count']] = clf.get_expanded_outrage_count()
    df.iloc[ix, col_map['valence']] = clf.get_valence_score()
    df.iloc[ix, col_map['arousal']] = clf.get_arousal_score()
    df.iloc[ix, col_map['sentiment']] = clf.get_sentiment_score()
    #df.iloc[ix, col_map['emoji_count']] = clf.get_emoji_count()
    
    topic_arousal_score = 0
    topic_valence_score = 0
    for tup in clf.get_topics():
        df.iloc[ix, col_map['topics']].append(tup)
        topic_arousal_score += clf.topic_arousal_scaler[tup[0]] * tup[1]
        topic_valence_score += clf.topic_valence_scaler[tup[0]] * tup[1]
    df.iloc[ix, col_map['topic_arousal']] = topic_arousal_score
    df.iloc[ix, col_map['topic_valence']] = topic_valence_score

    #df.iloc[ix, col_map['outrage']] = clf.get_outrage_score()
    '''    
    topic_arousal_score = 0
    topic_valence_score = 0
    for tup in clf.get_topics():
        df.iloc[ix, col_map['topics']].append(tup)
        topic_arousal_score += clf.topic_arousal_scaler[tup[0]] * tup[1]
        topic_valence_score += clf.topic_valence_scaler[tup[0]] * tup[1]
    
    list_insert = [
        clf.get_arousal_score(),
        clf.get_valence_score(),
        clf.get_sentiment_score(),
        clf.get_topics(),
        topic_arousal_score,
        topic_valence_score
    ]
    to_insert.append(list_insert)
    
to_insert = np.array(to_insert)

Processing tweet number 1, time elapsed: 0:00:00.000247
Processing tweet number 2501, time elapsed: 0:09:13.622449
Processing tweet number 5001, time elapsed: 0:18:06.283842
Processing tweet number 7501, time elapsed: 0:27:13.394743
Processing tweet number 10001, time elapsed: 0:36:24.396497
Processing tweet number 12501, time elapsed: 0:44:54.015470
Processing tweet number 15001, time elapsed: 0:53:54.847827


KeyboardInterrupt: 

In [28]:
arr_insert = np.array(to_insert)
arr_insert.shape

(17199, 5)

In [32]:
# DON'T DO THIS ONE -- I FORGOT TO ADD THE TOPICS 
for index in np.arange(arr_insert.shape[0]):
    df.iloc[index,col_map['arousal']] = arr_insert[index,0]
    df.iloc[index,col_map['valence']] = arr_insert[index,1]
    df.iloc[index,col_map['sentiment']] = arr_insert[index,2]
    df.iloc[index,col_map['topic_arousal']] = arr_insert[index,3]
    df.iloc[index,col_map['topic_valence']] = arr_insert[index,4]

df.head()

,text,topic,log_followers,log_retweets,engagement_flag,positive_emo,outrage_emo,net_emo_outrage,predicted_sentiment,negative_sentiment_prob,num_outrage_words_ext,num_outrage_words,arousal,valence,sentiment,topics,topic_arousal,topic_valence,outrage
user.description,,,,,,,,,,,,,,,,,,,
"Part of the #VastRightWingConspiracy -- thanks, @HillaryClinton. Better candidate: https://www.randpaul.com/ Retweeting not endorsement. Image: TV lover.","#BanTheBox means ""all is #forgiven""/NO MORE federal #NCIC checks when buying guns? Please consider it, Mr. President @POTUS. #forgiving",G,5.252273,0.0,0,0,0,0,4,0.265578,0,0,3.994752,4.674877,0.457199,"[(13, 0.131495800971), (27, 0.425939277591), (46, 0.0980153575283), (50, 0.266771786131), (13, 0.131394500046), (27, 0.425844825795), (46, 0.0980001633593), (50, 0.266982733022), (13, 0.1309867010...",1.144233,0.910757,0
War Veteran who loves his country. Never forget those who died in 'Benghazi'. 864th Engineer Battalion (Combat) (Heavy),BOYCOTT NOV 4 @ABC @CMAwards &gt; THREAT TO ALL GUN OWNERS - BOYCOTT HIS MUSIC &gt; @TheTimMcGraw GUN CONTROL EXTREMIST https://t.co/xMuE2o7vV7 …,G,9.936148,0.0,0,0,0,0,4,0.449931,0,0,5.726513,4.928830,0.486539,"[(6, 0.208727923843), (23, 0.0791057461022), (29, 0.0851475614148), (39, 0.391552329996), (50, 0.164953618131), (6, 0.208727689676), (23, 0.0791057173202), (29, 0.0851462087178), (39, 0.3915507425...",0.452195,0.388114,0
"Retired: cowboy,truck driver Served in the USAF, hobbies include Shooting sports and hunt and camp born in Calif.and sadly left in 1984","Savage: This is the Most Corrupt, Degenerate, Criminal Gov’t in American History https://t.co/qW12kzuBWH Take my guns lead first.",G,4.043051,0.0,0,0,0,0,0,0.583269,1,0,5.066761,4.056568,0.510775,"[(12, 0.13128764681), (27, 0.106913714881), (33, 0.325289099174), (50, 0.351661054287), (12, 0.131218108339), (27, 0.10687431201), (33, 0.325306845499), (50, 0.351752249303)]",0.561759,0.448320,0
Bartender...Bouncer...Autobody......Gravedigger.....Supermodal......OG.....Playboy.........:)\r\n\r\n\r\nDAM...NICE...GUY...MAFIA DEMOLITION INSTALATION,Guns N' Roses - November Rain https://t.co/w2pHuG0QBa via @YouTube,G,6.107023,0.0,0,0,0,0,4,0.470496,0,0,7.740000,3.660000,0.501845,"[(16, 0.190694556966), (29, 0.375986293763), (50, 0.274985815938), (16, 0.190722817926), (29, 0.376140201094), (50, 0.274803647647)]",0.366314,0.284348,0
Ming doesn't have a trajectory problem.,"I'll say it again: #Tarantino was merely correct. And, to his credit, he's sticking to his rhetorical guns. #BadCops https://t.co/7M4uTpXwyc",G,5.081404,0.0,0,0,0,0,4,0.362593,1,0,4.637737,4.979904,0.480766,"[(30, 0.223023678262), (49, 0.511015742518), (50, 0.186793912553), (30, 0.224087096145), (49, 0.511083915034), (50, 0.185662322154)]",0.508377,0.490181,0


In [52]:
df.iloc[18338,:]

text                       Liberals can't talk about evil, so they talk about guns @bayes46  https://t.co/wLCFQ4mslw
topic                                                                                                              G
log_followers                                                                                                5.80513
log_retweets                                                                                                       0
engagement_flag                                                                                                    1
positive_emo                                                                                                       0
outrage_emo                                                                                                        0
net_emo_outrage                                                                                                    0
predicted_sentiment                                             

In [53]:
df.to_csv('partially_measured.csv')

In [ ]:
# DO THIS ONE OR SOME OTHER WAY YOU LIKE
for index in np.arange(arr_insert.shape[0]):
    df.iloc[index,col_map['arousal']] = arr_insert[index,0]
    df.iloc[index,col_map['valence']] = arr_insert[index,1]
    df.iloc[index,col_map['sentiment']] = arr_insert[index,2]
    df.iloc[index,col_map['topics']] = arr_insert[index,3]
    df.iloc[index,col_map['topic_arousal']] = arr_insert[index,4]
    df.iloc[index,col_map['topic_valence']] = arr_insert[index,5]

df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler